In [4]:
from datetime import datetime, timedelta
import pandas as pd
import time
import calendar
from sqlalchemy import text, create_engine

In [5]:
SRC_DB_CONFIG = {
    'host': '138.68.88.175',
    'port': 5432,
    'database': 'csd_bi',
    'user': 'datalens_utl',
    'password': 'mQnXQaHP6zkOaFdTLRVLx40gT4'
}
TICKETS_DB_CONFIG = {
    'host': '138.68.88.175',
    'port': 5416,
    'database': 'ticket_replica',
    'user': 'klaksik77',
    'password': '6g3u0k13GhPhC2fvvPO'
}
DWH_DB_CONFIG = {
    'host': 'primarydwhcsd.aerxd.tech',
    'port': 6432,
    'database': 'postgres',
    'user': 'ste',
    'password': 'ILzAYQ72aEe9'
}

def _make_engine(db, include_options: bool = True):
    conn = f"postgresql+psycopg2://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['database']}"
    connect_args = {
        "connect_timeout": 10,
        "keepalives": 1,
        "keepalives_idle": 30,
        "keepalives_interval": 10,
        "keepalives_count": 5,
    }
    if include_options:
        connect_args["options"] = "-c statement_timeout=600000"
    return create_engine(
        conn,
        pool_pre_ping=True,
        pool_recycle=180,
        pool_size=5,
        max_overflow=2,
        connect_args=connect_args,
    )

ENGINE_SRC = _make_engine(SRC_DB_CONFIG, include_options=True)
ENGINE_TICKETS = _make_engine(TICKETS_DB_CONFIG, include_options=True)
ENGINE_DWH = _make_engine(DWH_DB_CONFIG, include_options=False)

In [6]:
def safe_parse_date(date_str):
    """Пытается создать дату, если день превышает допустимое — сдвигает на последний день месяца."""
    try:
        return datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        # Разбор вручную
        try:
            year, month, day = map(int, date_str.split('-'))
            last_day = calendar.monthrange(year, month)[1]
            return datetime(year, month, last_day)
        except Exception as e:
            print(f"Ошибка при корректировке даты: {e}")
            raise

def get_invoice_data_by_days(start_date, end_date, engine):
    try:
        start = safe_parse_date(start_date)
        end = safe_parse_date(end_date)
    except Exception:
        return pd.DataFrame()

    if start > end:
        print("Дата начала позже даты окончания.")
        return pd.DataFrame()

    final_df = pd.DataFrame()
    current_date = start
    day_counter = 1

    while current_date <= end:
        day_start = current_date.strftime('%Y-%m-%d 00:00:00')
        day_end = current_date.strftime('%Y-%m-%d 23:59:59')

        print(f"День {day_counter}: {current_date.strftime('%Y-%m-%d')}")

        query_get_data = f"""
            SELECT
                SUM(i.amount) FILTER (WHERE i.status_id = 2) as amount_success,
                SUM(i.amount) FILTER (WHERE i.status_id != 2) as amount_failed,
                i.client_id,
                lc.name as client_name,
                lcl.system_name as currency
            FROM orders.invoice i
                JOIN clients.client lc on lc.id = i.client_id
                JOIN lists.currency_list lcl on lcl.id = i.currency_id
            WHERE i.created_at >= '{day_start}' AND i.created_at <= '{day_end}'
            group by i.client_id, lc.name, lcl.system_name
            """
        try:
            query = text(query_get_data)  # оберни строку в text()
            daily_data = pd.read_sql(query, engine, params={})
            # daily_data = daily_data.drop(columns=['created_at'])
            if not daily_data.empty:
                daily_data['processing_date'] = current_date.strftime('%Y-%m-%d')
                final_df = pd.concat([final_df, daily_data], ignore_index=True)
                print(f"Размер final_df - {len(final_df)}")
                print(f"Получено {len(daily_data)} записей")
                # print(f"Уникальных плательщиков: {daily_data['payer_id'].nunique()}")
            else:
                print(f"Нет данных")
        except Exception as e:
            print(f"Ошибка при получении данных за {current_date.strftime('%Y-%m-%d')}: {e}")

        current_date += timedelta(days=1)
        day_counter += 1
        time.sleep(0.5)

    return final_df


# Использование
# data = get_invoice_data_by_days('2025-06-01', '2025-08-31', ENGINE_SRC)
data = pd.read_csv('data_invoices.csv')

In [7]:
# data.to_csv('data_invoices.csv', index=False)

In [11]:
query = """
    select 
        ticket_info ->> 'clientID' as client_id,
        case when ticket_info ->> 'appealDeclineReasonID' is null then 'success' else 'reject' end as failed_flg,
        ticket_info ->> 'orderID' as order_id
    from public.ticket
    where status_id = 4 and assigned_to <> 0 and
        created_at >= '2025-06-01' and created_at <= '2025-08-31' and ticket_info ->> 'clientID' <> '0'
"""
data_tickets = pd.read_sql(query, ENGINE_TICKETS, params={})

In [12]:
data_tickets

,client_id,failed_flg,order_id
0,3829,reject,176100100
1,3864,success,185606850
2,3829,success,176158309
3,3829,success,185638479
4,3216,success,174739830
...,...,...,...
48711,3862,success,174642301
48712,3829,success,174670598
48713,2697,success,176233154
48714,3845,success,174688909


In [31]:
query_error_tickets = f"""
    select 
        error_code,
        id as order_id,
        CASE
        WHEN i.status_id <> 2 THEN
        CASE
            WHEN i.error_code = 101 THEN 'Rejected by client blacklist rule'
            WHEN i.error_code = 102 THEN 'From merchant ID block'
            WHEN i.error_code = 103 THEN 'Suspicious LLC country'
            WHEN i.error_code = 104 THEN 'Suspicious provider'
            WHEN i.error_code = 105 THEN 'Suspicious subnet'
            WHEN i.error_code = 107 THEN 'Blacklist API'
            WHEN i.error_code = 108 THEN 'Blacklist Form'
            WHEN i.error_code = 109 THEN 'Limit exceeded'
            WHEN i.error_code = 111 THEN 'Bad user agent'
            WHEN i.error_code = 112 THEN 'Primary traffic blocked'
            WHEN i.error_code = 114 THEN 'Card blocked'
            WHEN i.error_code = 115 THEN 'Canceled by peer'

            WHEN i.error_code = 2000 THEN 'Cannot accept default terminal ID'
            WHEN i.error_code = 2002 THEN 'Another terminal ID was used'
            WHEN i.error_code = 2003 THEN 'Not supported currency'
            WHEN i.error_code = 2004 THEN 'Not supported payment system'
            WHEN i.error_code = 2007 THEN 'Not supported bank'
            WHEN i.error_code = 2010 THEN 'Invalid order status'
            WHEN i.error_code = 2033 THEN 'Method ID validation failed'
            WHEN i.error_code = 2034 THEN 'Merchant URL validation failed'
            WHEN i.error_code = 2037 THEN 'Can refund only H2H'
            WHEN i.error_code = 2038 THEN 'Order is already refunded'
            WHEN i.error_code = 2041 THEN 'Refund timeout exceeded'
            WHEN i.error_code = 2042 THEN 'Engine not set'
            WHEN i.error_code = 2044 THEN 'Engine order ID not set'
            WHEN i.error_code = 2045 THEN 'Balance validation fail'
            WHEN i.error_code = 2048 THEN 'Can chargeback only H2H'
            WHEN i.error_code = 2049 THEN 'Can not change success refund'
            WHEN i.error_code = 3001 THEN 'Blocked by limit'
            WHEN i.error_code = 3002 THEN 'Unknown external error'
            WHEN i.error_code = 3007 THEN '3DS PARes status not Y'
            WHEN i.error_code = 3008 THEN '3DS VERes status U'
            WHEN i.error_code = 3009 THEN '3DS decline in VERes'
            WHEN i.error_code = 3010 THEN 'No attempts left'
            WHEN i.error_code = 3011 THEN 'Session time limit'
            WHEN i.error_code = 3013 THEN 'TDS signature test failed'
            WHEN i.error_code = 3016 THEN 'Fraud do not honour'
            WHEN i.error_code = 3017 THEN 'Original transaction not found'
            WHEN i.error_code = 3018 THEN 'Card limits exceeded'
            WHEN i.error_code = 3019 THEN 'Wrong expiry date'
            WHEN i.error_code = 3025 THEN 'Security violation'
            WHEN i.error_code = 3026 THEN 'Transaction not permitted'
            WHEN i.error_code = 3031 THEN 'Cannot contact bank'
            WHEN i.error_code = 3032 THEN 'Invalid operation'
            WHEN i.error_code = 3033 THEN 'SSL forbidden'
            WHEN i.error_code = 3036 THEN 'Refund more than deposited'
            WHEN i.error_code = 3037 THEN '3DS rule error'
            WHEN i.error_code = 3038 THEN 'Terminal select rule error'
            WHEN i.error_code = 3039 THEN 'General external error'
            WHEN i.error_code = 3040 THEN '3DS declined in ARes'
            WHEN i.error_code = 3042 THEN '3DS authentication cancelled'
            WHEN i.error_code = 3043 THEN '3DS authentication failed'
            WHEN i.error_code = 3044 THEN '3DS unknown RReq status'
            WHEN i.error_code = 3045 THEN '3DS erroneous challenge'
            WHEN i.error_code = 3047 THEN 'Postal address verification failed'
            WHEN i.error_code = 3052 THEN 'Duplicate order'
            WHEN i.error_code = 3057 THEN 'Invalid CVV2'
            WHEN i.error_code = 3058 THEN 'Invalid AVS'
            WHEN i.error_code = 3061 THEN 'Currency mismatch'
            WHEN i.error_code = 3062 THEN 'Authorization expired'
            WHEN i.error_code = 3063 THEN 'Reversal needed'
            WHEN i.error_code = 3064 THEN 'Invalid transaction type'
            WHEN i.error_code = 3065 THEN 'Batch not found'
            WHEN i.error_code = 3066 THEN 'Settlement failed'
            WHEN i.error_code = 3067 THEN 'Capture amount exceeded'
            WHEN i.error_code = 3069 THEN 'Currency not accepted'
            WHEN i.error_code = 3070 THEN 'Card scheme restriction'
            WHEN i.error_code = 3071 THEN 'Merchant category restriction'
            WHEN i.error_code = 3075 THEN 'Recurring cancelled'
            WHEN i.error_code = 3076 THEN 'Recurring expired'
            WHEN i.error_code = 3077 THEN 'Recurring not found'
            WHEN i.error_code = 3079 THEN 'Velocity check failed'
            WHEN i.error_code = 3083 THEN 'Invalid token'
            WHEN i.error_code = 3085 THEN 'Account verification needed'
            WHEN i.error_code = 3087 THEN 'Risk check failed'
            WHEN i.error_code = 3088 THEN 'PCI compliance failure'
            WHEN i.error_code = 3101 THEN 'Invalid billing descriptor'
            WHEN i.error_code = 3102 THEN 'Dynamic currency conversion failed'
            WHEN i.error_code = 3104 THEN 'Currency conversion expired'
            WHEN i.error_code = 3105 THEN 'Cross-border declined'
            WHEN i.error_code = 3108 THEN 'Mandate required'
            WHEN i.error_code = 3109 THEN 'Mandate invalid'
            WHEN i.error_code = 3110 THEN 'Direct debit failed'
            WHEN i.error_code = 3111 THEN 'Chargeback dispute lost'
            WHEN i.error_code = 3112 THEN 'Fraud investigation'
            WHEN i.error_code = 3114 THEN 'Account temporarily locked'
            WHEN i.error_code = 3117 THEN 'Payment method invalid'
            WHEN i.error_code = 3118 THEN 'Merchant account disabled'

            WHEN i.error_code IN (2013, 3091) THEN 'Client Amount Below Limit'
            WHEN i.error_code IN (2014, 3092) THEN 'Client Amount Over Limit'
            WHEN i.error_code IN (2015, 3093) THEN 'Client Turnover Below Limit'
            WHEN i.error_code IN (2016, 3094) THEN 'Client Turnover Over Limit'
            WHEN i.error_code IN (2019, 3097) THEN 'Engine Amount Below Limit'
            WHEN i.error_code IN (2020, 3098) THEN 'Engine Amount Over Limit'
            WHEN i.error_code IN (2021, 3099) THEN 'Engine Turnover Below Limit'
            WHEN i.error_code IN (2022, 3100) THEN 'Engine Turnover Over Limit'
            WHEN i.error_code = 2047 THEN 'Cannot chargeback unsuccessful order'
            WHEN i.error_code = 3035 THEN '3DS rule failed'
            WHEN i.error_code = 3089 THEN 'Card validation error'
            WHEN i.error_code = 2012 THEN 'Payment form already generated'
            WHEN i.error_code = 3107 THEN 'This feature is coming soon, just wait for it'
            WHEN i.error_code = 2035 THEN 'Invalid phone number'
            WHEN i.error_code = 106 THEN 'Rejected by anti-fraud rule'
            WHEN i.error_code = 3015 THEN 'Invalid merchant'
            WHEN i.error_code = 3020 THEN 'Contact issuer'
            WHEN i.error_code = 3051 THEN 'Try again later'
            WHEN i.error_code = 3055 THEN 'CVV2 failure'
            WHEN i.error_code = 3091 THEN 'Amount is below limit'
            WHEN i.error_code = 2032 THEN 'Payment system validation failed'
            WHEN i.error_code = 2036 THEN 'Fail on engine action'
            WHEN i.error_code = 3023 THEN 'Wrong PAN'
            WHEN i.error_code = 2014 THEN 'Amount is over limit'
            WHEN i.error_code = 4001 THEN 'Bank validation failed'

            WHEN i.error_code = 113 THEN 'Requisites are broken'
            WHEN i.error_code = 1001 THEN 'Internal error'
            WHEN i.error_code = 1003 THEN 'Internal error'
            WHEN i.error_code = 2027 THEN 'IP validation failed'
            WHEN i.error_code = 3103 THEN 'Card number validation failed'
            WHEN i.error_code = 1000 THEN 'Internal error'
            WHEN i.error_code = 2001 THEN 'Terminal ID credentials not found'
            WHEN i.error_code = 3003 THEN 'SSL error'
            WHEN i.error_code = 3056 THEN 'Transaction not permitted to cardholder'
            WHEN i.error_code = 3074 THEN '3Dv2 failed'
            WHEN i.error_code = 2016 THEN 'Turnover is over limit'
            WHEN i.error_code = 3096 THEN 'BIN validator failed'
            WHEN i.error_code = 3113 THEN 'Invalid phone number'
            WHEN i.error_code = 3120 THEN 'Bad request. Check your credentials'
            WHEN i.error_code = 3121 THEN 'Card declined by payment system'
            WHEN i.error_code = 3005 THEN '3DS v1 authentication is not possible. Declined by iReq in PARes'
            WHEN i.error_code = 3024 THEN 'Not enough funds'
            WHEN i.error_code = 3073 THEN '3DS additional authentication required'
            WHEN i.error_code = 3081 THEN 'Card not registered for 3DS'
            WHEN i.error_code = 2017 THEN 'PAN validator failed'
            WHEN i.error_code = 2029 THEN 'This feature is coming soon, just wait for it'
            WHEN i.error_code = 3006 THEN '3DS v1 authentication is not possible. Declined. DS connection timeout'
            WHEN i.error_code = 3014 THEN 'TDS impossible'
            WHEN i.error_code = 3028 THEN 'Invalid card number'
            WHEN i.error_code = 3046 THEN '3DS authentication. ACS cancelled challenge by timeout'
            WHEN i.error_code = 3053 THEN 'Payer country is forbidden'
            WHEN i.error_code = 3090 THEN 'Payment form already generated'
            WHEN i.error_code = 2018 THEN 'BIN validator failed'
            WHEN i.error_code = 2031 THEN 'Bank validation failed'
            WHEN i.error_code = 3122 THEN 'Invalid amount or currency'
            WHEN i.error_code = 2039 THEN 'Cannot refund unsuccessful order'
            WHEN i.error_code = 3012 THEN '3DS authentication is not possible'
            WHEN i.error_code = 3041 THEN '3DS authentication status in ARes is unknown'
            WHEN i.error_code = 3086 THEN 'Cannot get order by UID'
            WHEN i.error_code = 2022 THEN 'Turnover is over limit'
            WHEN i.error_code = 2046 THEN 'Order is already chargebacked'
            WHEN i.error_code = 3060 THEN 'Non-3DS operation is not allowed'
            WHEN i.error_code = 3072 THEN 'Cannot find gate for card'
            WHEN i.error_code = 3095 THEN 'PAN validator failed'
            WHEN i.error_code = 2025 THEN 'Card number validation failed'

            WHEN i.error_code = 101 THEN 'Rejected by anti-fraud rule'
            WHEN i.error_code = 110 THEN 'Requests limit exceeded. User has been blacklisted'
            WHEN i.error_code = 3080 THEN '3DS declined'
            WHEN i.error_code = 2008 THEN 'Cannot get order by UID'
            WHEN i.error_code = 3093 THEN 'Turnover is below limit'
            WHEN i.error_code = 2024 THEN 'BIN validator failed'
            WHEN i.error_code = 103 THEN 'Suspicious request'
            WHEN i.error_code = 2050 THEN 'Cannot update chargeback to new status'
            WHEN i.error_code = 3021 THEN 'Wrong merchant ID/terminal ID'
            WHEN i.error_code = 3029 THEN 'Reenter transaction'
            WHEN i.error_code = 3082 THEN 'Transaction expired'
            WHEN i.error_code = 2006 THEN 'Insufficient funds'
            WHEN i.error_code = 3034 THEN 'SSL without CVC forbidden'
            WHEN i.error_code = 2028 THEN 'Not configured for you, please contact the manager'
            WHEN i.error_code = 3106 THEN 'Not configured for you, please contact the manager'
            WHEN i.error_code = 3119 THEN 'Not found. Check your credentials'
            WHEN i.error_code = 2043 THEN 'Engine not available'
            WHEN i.error_code = 3030 THEN 'Invalid message format'
            WHEN i.error_code = 3078 THEN 'Payment rejected'
            WHEN i.error_code = 3084 THEN 'Transaction timed out'
            WHEN i.error_code = 3092 THEN 'Amount is over limit'
            WHEN i.error_code = 3097 THEN 'Amount is below limit'
            WHEN i.error_code = 3115 THEN 'Contact support'
            WHEN i.error_code = 3048 THEN 'General payment system error'
            WHEN i.error_code = 3054 THEN '3DS timeout'
            WHEN i.error_code = 3059 THEN 'Card not active'
            WHEN i.error_code = 3068 THEN 'Transaction not allowed at terminal'
            WHEN i.error_code = 2030 THEN 'Geo validation failed'
            WHEN i.error_code = 3116 THEN 'Cardholder name not specified'
            WHEN i.error_code = 4000 THEN 'Geo validation failed'
            WHEN i.error_code = 1002 THEN 'Internal error'
            WHEN i.error_code = 2005 THEN 'No available engines'
            WHEN i.error_code = 2040 THEN 'Order is already on refund'

            WHEN fail_cause LIKE '%temporarily blocked for%' THEN 'Temporarily blocked for exceeded spam'
            WHEN fail_cause LIKE '%requests limit exceeded%' THEN 'Requests limit exceeded'
            WHEN fail_cause LIKE '%requests limit exceeded hardly%' THEN 'Requests limit exceeded hardly. User has been added to black-list'
            WHEN fail_cause LIKE '%temporarily blocked for exceeded card spam%' THEN 'Temporarily blocked for exceeded card spam filter limits '
            WHEN fail_cause LIKE '%ip is in API blacklist%' THEN 'IP is in API Blacklist'
            WHEN fail_cause LIKE '%suspicious request%' THEN 'Suspicious Request'
            WHEN fail_cause LIKE '%primary traffic blocked%' THEN 'Primary traffic blocked'
            WHEN fail_cause IS NULL AND i.error_code IS NULL and status_id = 5 then 'No available engines'
            WHEN fail_cause is NULL and i.error_code IS NULL and status_id = 4 then 'Internal Error'
            WHEN fail_cause LIKE '%ip is in Form blacklist%' THEN 'IP is in Form Blacklist'
            WHEN i.error_code = 2009 THEN 'Order Expired'
            when status_id in (3, 4, 5) and i.engine_id is null then 'Unknown Reason'
            else 'not failed'
            END
            ELSE NULL
        END AS failed_cause
    from orders.invoice i
    where i.status_id <> 2 and i.id = ANY(:order_ids) and error_code <> '0'
"""
ids = data_tickets['order_id'].astype(int).tolist()
data_error_tickets = pd.read_sql(text(query_error_tickets), ENGINE_SRC, params={"order_ids": ids})

In [33]:
data_error_tickets

,error_code,order_id,failed_cause
0,1000,95370064,Internal error
1,1000,96049406,Internal error
2,1000,96601021,Internal error
3,1000,103527411,Internal error
4,1000,106339946,Internal error
...,...,...,...
7022,1000,197396625,Internal error
7023,2036,197400354,Fail on engine action
7024,2005,197411508,No available engines
7025,2036,197432088,Fail on engine action


In [36]:
data_tickets['order_id'] = data_tickets['order_id'].astype(str)
data_error_tickets['order_id'] = data_error_tickets['order_id'].astype(str)
data_tickets = data_tickets.merge(data_error_tickets, on='order_id', how='left')

In [42]:
data_tickets[data_tickets['failed_flg'] == 'reject']['failed_cause'].value_counts()

failed_cause
Internal error                                            5632
Order Expired                                              381
No available engines                                       296
Canceled by peer                                           150
IP is in Form Blacklist                                    126
Fail on engine action                                      115
Temporarily blocked for exceeded spam                       14
Requests limit exceeded                                     10
Invalid token                                                9
Fraud do not honour                                          3
Client Amount Below Limit                                    3
IP is in API Blacklist                                       3
Insufficient funds                                           2
not failed                                                   2
Transaction timed out                                        2
Contact issuer                            